¿Qué es una corrutina?

Una corrutina es una función que se puede suspender su ejecución y posteriormente restaurarla. En estas paradas puede haber un intercambio de datos en ambos sentidos. A diferencia de los generadores, aquí hay transmisión de información también de la función base a la corrutina.

Ejemplo: se pretende crear una funcion que compruebe si un valor de presion de un motor es alto o bajo. Pra poder configurar el sistema de medicion se requiere hacer uso de una funcion llamada activar_medidor, la cual tarda 10 segundos en configurar el medidor. Se pueden realizartantas operaciones como sean necesarias medir, pero al terminar de usar el medidor hay que desactivarlo usando la funcion desactivar_medidor, la cual tarda 5 segundos en realizar la tarea. La presion actual del sistema la devuelve una funcion denomianda valor_presion_actual.

In [25]:
#Forma convencional sin uso corrutina
from time import sleep, time
import random

In [10]:
def activar_medidor(minimo, maximo):
    """ Esta funcion simula que en un caso real se conectaria con
    el sistema, bases de datos, utilizaria protocolos de conexion, intercambio de informacion, etc """
    valor_medio = (minimo + maximo) / 2
    sleep(10)
    print("Medidor activado!")
    return valor_medio

def desactivar_medidor():
    """Esta funcion simula el tiempo empleado en desmantelar el sistem de comprobacion"""
    sleep(5)
    print("Medidor desactivado")
    return True

def valor_presion_actual(minimo = 1, maximo = 100):
    return random.choice(range(minimo, maximo))

#La funcion solicitada en el ejercicio de ejemplo se podria hacer como sigue utilizando funciones normales de python

def funcion_comprobar_presion(valor_a_probar, minimo=1, maximo=100):
    valor_medio = activar_medidor(minimo, maximo)
    if valor_a_probar > valor_medio:
        print(f'La presion es alta {valor_a_probar}')
    if valor_a_probar < valor_medio:
        print(f'La presion es baja {valor_a_probar}')
    if valor_a_probar == valor_medio:
        print(f'La presion es normal {valor_a_probar}')
    desactivar_medidor()


In [12]:
funcion_comprobar_presion(45, 20, 80)

Medidor activado!
La presion es baja 45
Medidor desactivado


In [13]:
#Ahora usando corrutinas 

In [28]:
def corrutina_comprobar_presion(minimo=1, maximo=100):
    valor_medio = activar_medidor(minimo, maximo)
    try:
        while True:
            valor_a_probar = yield 'Corrutina inicializada'
            if valor_a_probar > valor_medio:
                print(f'La presion es alta {valor_a_probar}')
            if valor_a_probar < valor_medio:
                print(f'La presion es baja {valor_a_probar}')
            if valor_a_probar == valor_medio:
                print(f'La presion es normal {valor_a_probar}')
    except KeyboardInterrupt:
        desactivar_medidor()
        yield True
    except GeneratorExit:
        desactivar_medidor()

In [29]:
#ahora pasemos a ver la eficiencia de cino mediciones utilizando cada una de las funciones:

In [30]:
if __name__ == '__main__':
    inicio = time()
    for _ in range(5):
        v_actual = valor_presion_actual()
        funcion_comprobar_presion(v_actual)
    tardanza = time() - inicio
    print(f'Tiempo empleado: {tardanza:.3}s')
    
    inicio = time()
    corrutina = corrutina_comprobar_presion()
    print(next(corrutina))
    inicializar = time() - inicio
    print(f'Tiempo empleado para incializar {inicializar:.3}s')
    for _ in range(5):
        v_actual = valor_presion_actual()
        corrutina.send(v_actual)
    corrutina.throw(KeyboardInterrupt, 'saliendo de la ejecucion') #se podria usar tambien corrutina.close()
    tardanza = time() - inicio
    print(f'Tiempo empleado {tardanza:.3}s')

Medidor activado!
La presion es alta 69
Medidor desactivado
Medidor activado!
La presion es baja 11
Medidor desactivado
Medidor activado!
La presion es baja 32
Medidor desactivado
Medidor activado!
La presion es alta 97
Medidor desactivado
Medidor activado!
La presion es baja 35


Exception ignored in: <generator object corrutina_comprobar_presion at 0x0000023AB3A9CDD0>
Traceback (most recent call last):
  File "C:\Users\Arbusta\AppData\Local\Temp\ipykernel_1320\1349639084.py", line 12, in corrutina_comprobar_presion
NameError: name 'KeyBoardInterrupt' is not defined


Medidor desactivado
Tiempo empleado: 75.1s
Medidor activado!
Corrutina inicializada
Tiempo empleado para incializar 10.0s
La presion es baja 2
La presion es baja 17
La presion es baja 6
La presion es baja 40
La presion es baja 20
Medidor desactivado
Tiempo empleado 15.0s


Como se puede ver, existe una gran diferencia en el timepo de ejecucio entre la version con funciones normales y la version que utiliza corrutinas.

La principal ventaja de las corrutinas es que el contexto se mantiene estático dentro de la funcion generadora y, por tanto, se puede seguir haciendo  uso del medidor sin ningun problema. Para detener la ejecucion y cerrar el generador se utiliza la funcion *close*, que es manajda por la funcion generadora para desactivar la medicion. Adicionalmente, en ese ejemplo se maneja la excepcion producida por un *KeyBoardInterrupt*, lo que permite poder detener la ejecucion de forma controlada utilizando la funcion *throw*